<a href="https://colab.research.google.com/github/dsethia1/Snake_Ladder/blob/main/Snakes_Ladders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
%%writefile main.cpp
//Above line adds a file .cpp, where c++ code is run
#include <iostream>
#include <algorithm>
#include <map>
#include <vector>
#include <ctime>
#include <cmath>
using namespace std;

//@param gives the amount of time to delay the program for
int delay(int nanoseconds)
  {
     clock_t goal = nanoseconds + clock();
     //clock_t is an arithmetic type (usually an integer) capable of representing the process running time
     while(goal>clock());
     //clock() is a program that tracks the number of clock ticks since the program started
     return 1;
  }

//@param decides the position of the player
void output_move(int i){
  if(i < 0){ //moving backward
    cout << ". You got a snake and moved down " << -1*i << " steps." << endl;
  }
  else if (i > 0){ //moving forward
    cout << ". You got a ladder and moved up " << i << " steps." << endl;
  }
  else{ //player does not move
    cout << ". Nothing!" << endl;
  }
}

//@param snakes creates corresponding action for position; player on snake pos
//@param ladders creates corresponding action for position; player on ladder pos
void print_board(const map<int, int>& snakes, const map<int, int>& ladders, int position[], int players) {
    int board_size = 10;
    vector<string> board(pow(board_size,2) + 1, "   ");

    for (int i = 1; i <= pow(board_size,2); ++i){
        //add position as s or l in board if snake or ladder is there
        if(snakes.count(i)) {
            board[i] = " S↓";
        }
        else if(ladders.count(i)){
            board[i] = " L↑";
        }
        else{
            if(i < 10){
              board[i] = "  " + to_string(i);
            }
            else{
              board[i] = " " + to_string(i);
            }
            //if not snake/ladder display number, adding i as a string
            //if i<10 do 2 spaces, otherwise 1 space.
        }
    }
    //replace all positions with players with the player icon
    for(int i = 0; i<players; i++){
      board[position[i]] = " P" + to_string(i+1);
    }
    //begin checking board from top, go down in a zigzag pattern
    for (int row = board_size - 1; row >= 0; row--) {
        for (int col = 0; col < board_size; col++) {
            int index = 0;
            if(row % 2 == 1){
              index = row * board_size + (col+1);
              //odd rows go left to right
            }
            else{
              index = (row+1) * board_size - col;
              //even rows go right to left
            }
            cout << board[index] << "   ";
            //print board
        }
        cout << endl;
        cout << endl;
    }
}

int main() {
    int nanoseconds = 1000000;
    srand(time(0));
    //beginning the game
    cout << "Hello! Welcome to Snakes and Ladders: Mystery Edition." << endl;
    cout << "How many players will be playing? Please choose a number between 2-4!" << endl << endl;
    map<int, int> board;
    map<int,int> snakes;
    map<int, int> ladders;
    int player;
    cin >> player;
    while(cin.fail()){ //in case player enters incompatible input
        cin.clear();
        cin >> player;
    }
    cout << "Perfect! "<< player << " players it is!" << endl;
    delay(nanoseconds);
    cout << "Please enter the names of the players. " << endl << endl;
    string players_names[player];
    for(int i = 0; i < player; i++){
      cout << "Player " << i + 1 << ": ";
      cin >> players_names[i];
    }
    cout << "Hi, players! You'll start by rolling a die to see which player's turn is first. " << endl;
    cout << "If there is a tie, the player initialized before will go first. " << endl << endl;
    delay(nanoseconds);
    cout << "Please press any key to roll the dice. " << endl;
    pair<int, int> order[player]; //keeping track of players rolls to determine order of turns
    string turn;
    for(int i = 0; i < player; i++){
      cin >> turn;
      int move = rand() % 6 + 1;
      cout << players_names[i] << " rolled a " << move << "!" << endl << endl; //display to users
      delay(nanoseconds);
      order[i].first = 6 - move;
      order[i].second = i;
    }
    sort(order, order + player);
    for(int i = 0; i < player; i++){ //print out the order of play for all players to see
      cout << players_names[order[i].second] << " will go " << i+1 << " in turn!" << endl << endl;
      delay(nanoseconds);
    }
    int position[player];
    for(int i = 0; i < player; i++){ //set all players to position 0 prior to start of game
      position[i] = 0;
    }
    int check = 0;
    print_board(snakes, ladders, position, player); //print the board
    while(check == 0){
      for(int i = 0; i < player; i++){ //each iteration of this loop is one player's turn
        cout << players_names[order[i].second] << ", press any key to roll the dice." << endl;
        cin >> turn;
        int move = rand() % 6 + 1; //generate the dice result
        cout << players_names[order[i].second] << " rolled a " << move << "!" << endl << endl; //show player what they rolled
        delay(nanoseconds);
        if(move + position[order[i].second] > 100){ //in case the move propels the player past position 100
          cout << "Invalid move" << endl;
          delay(nanoseconds);
        }
        else{ //if all is well, execute the turn.
          position[order[i].second] += move;
          cout << "Position: " << position[order[i].second];
          if(board.count(position[order[i].second]) > 0){
            position[order[i].second] += board[position[order[i].second]];
            output_move(board[position[order[i].second]]);
            //position already defined
          }
          else{
            int board_move = rand() % 15 + 5;
            int snake_or_ladder = rand() % 3;
            if(snake_or_ladder == 0){
              //invalidate the snake roll if in the first ten spaces
              if(position[order[i].second] <= 10){
                board[position[order[i].second]] = 0;
              }
              else{
                board_move = -1 * board_move;
                board_move = max(board_move, (-1 * position[order[i].second]));
                board[position[order[i].second]] = board_move;
                snakes[position[order[i].second]] = board_move;
              }
            }
            else if (snake_or_ladder == 1){
              //invalidate the ladder roll if in the last ten spaces
              if(position[order[i].second] >= 90){
                board[position[order[i].second]] = 0;
              }
              else{
                board_move = min(board_move, (100-position[order[i].second]));
                board[position[order[i].second]] = board_move;
                ladders[position[order[i].second]] = board_move;
              }
            }
            else{
              board[position[order[i].second]] = 0;
            }
            output_move(board[position[order[i].second]]);
            position[order[i].second] += board[position[order[i].second]];
          }
          delay(nanoseconds);
          cout << "New position: " << position[order[i].second] << endl << endl;
          delay(nanoseconds);
        }
        print_board(snakes, ladders, position, player);
        if(position[order[i].second] == 100){ //if the player whose turn it is has reached a 100, declare them the winner and stop the game
          print_board(snakes, ladders, position, player);
          cout << players_names[order[i].second] << " wins!" << endl << endl;
          check = 1; //the variable in the while loop that perpetuates the game
          break;
        } //end of (check for position 100) loop
      } //end of player turn loop
    } //end of large game while loop
    return 0;
}




Overwriting main.cpp


In [12]:
!g++ main.cpp -o main
#Runs the file main.cpp through g++ compiler (c++)

In [13]:
!./main
#Calls main file and runs it

Hello! Welcome to Snakes and Ladders: Mystery Edition.
How many players will be playing? Please choose a number between 2-4!

2
Perfect! 2 players it is!
Please enter the names of the players. 

Player 1: Apple
Player 2: Bananan
Hi, players! You'll start by rolling a die to see which player's turn is first. 
If there is a tie, the player initialized before will go first. 

Please press any key to roll the dice. 
a
Apple rolled a 6!

b
Bananan rolled a 3!

Apple will go 1 in turn!

Bananan will go 2 in turn!

 91    92    93    94    95    96    97    98    99    100   

 90    89    88    87    86    85    84    83    82    81   

 71    72    73    74    75    76    77    78    79    80   

 70    69    68    67    66    65    64    63    62    61   

 51    52    53    54    55    56    57    58    59    60   

 50    49    48    47    46    45    44    43    42    41   

 31    32    33    34    35    36    37    38    39    40   

 30    29    28    27    26    25    24    23    22